In [1]:
import os
import time
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch

import viz
import utils
from preprocess import get_preprocess
from explainer.sparse import SparseExplainer

image_path = 'images/elephant.png'
image_class = 101  # tusker
raw_img = viz.pil_loader(image_path)
plt.figure(figsize=(5, 5))
plt.imshow(raw_img)
plt.axis('off')
plt.title('class_id=101 (Tusker)')

model_name = 'resnet50'
method_name = 'sparse'
show_style = 'imshow'
transf = get_preprocess(model_name, method_name)
model = utils.load_model(model_name)
model.cuda()

/fs/clip-ml/shifeng/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [2]:
explainer = SparseExplainer(model)
        
inp = transf(raw_img)
inp = utils.cuda_var(inp.unsqueeze(0), requires_grad=True)
saliency = explainer.explain(inp)
saliency = utils.upsample(saliency, (raw_img.height, raw_img.width))
saliency_map = saliency.cpu().numpy()
    
plt.figure(figsize=(25, 15))
plt.imshow(raw_img)
plt.axis('off')
plt.title('class_id=101 (Tusker)')
if show_style == 'camshow':
    viz.plot_cam(np.abs(saliency).max(axis=1).squeeze(),
                 raw_img, 'jet', alpha=0.5)
else:
    saliency = saliency.squeeze().transpose(1, 2, 0)
    saliency -= saliency.min()
    saliency /= (saliency.max() + 1e-20)
    plt.imshow(saliency, cmap='gray')

plt.axis('off')
# plt.tight_layout()
# plt.savefig('images/tusker_saliency.png')

RuntimeError: t() expects a 2D tensor, but self is 4D